<a href="https://colab.research.google.com/github/shuen1106lu/Program-Language/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 授權 Google Colab 存取 Google Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd

In [ ]:
# 認證 Google Sheets API
creds, _ = default()
gc = gspread.authorize(creds)

# 讀取 Google Sheets(病人資料)
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ucZMuaJx7Y7yBj8fm2A0MK0WTU4RyEKlDiDzqlmQKcA/edit?usp=sharing')

# 選擇第一個工作表
worksheet = gsheets.get_worksheet(0)